In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import csv

# SeleniumのWebDriverをセットアップ
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 初期ページにアクセス
base_url = "https://biz-maps.com/search?sharingSearchHistoryId=895224"
driver.get(base_url)

# URLを格納するリスト
urls = []

# ページを移動するためのループ
for _ in range(250): #ページ数
    try:
        # 現在のURLを取得してリストに追加
        current_url = driver.current_url
        urls.append(current_url)

        # 次のページリンクを探してクリック
        next_button = driver.find_element(By.XPATH, "//a[@rel='next']")
        next_button.click()
        time.sleep(4)  # ページが完全にロードされるのを待つ
    except Exception as e:
        print("次のページが見つかりません:", e)
        break
    

# WebDriverを終了
driver.quit()

# CSVファイルに保存
csv_filename = "aiti5.csv"
with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["URL"])  # ヘッダー行
    for url in urls:
        writer.writerow([url])

print(f"URLリストを {csv_filename} に保存しました。")

次のページが見つかりません: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[@rel='next']"}
  (Session info: chrome=138.0.7204.93); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x00000001046fce6c cxxbridge1$str$ptr + 2722840
1   chromedriver                        0x00000001046f4d74 cxxbridge1$str$ptr + 2689824
2   chromedriver                        0x00000001042463ec cxxbridge1$string$len + 90648
3   chromedriver                        0x000000010428d544 cxxbridge1$string$len + 381808
4   chromedriver                        0x00000001042ce934 cxxbridge1$string$len + 649056
5   chromedriver                        0x0000000104281834 cxxbridge1$string$len + 333408
6   chromedriver                        0x00000001046bff88 cxxbridge1$str$ptr + 2473268
7   chromedriver                        0x00000001046c31f4 cxxbr

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import csv
# 開始時刻の記録
start_time = time.time()

# 入力ファイル名と出力ファイル名
input_csv = "aiti5.csv"
output_csv = "aiti5com.csv"

# ヘッダー情報を指定
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# 入力CSVからURLを読み込む
urls = []
with open(input_csv, mode="r", encoding="utf-8") as file:
    reader = csv.reader(file)
    next(reader)  # ヘッダーをスキップ
    urls = [row[0] for row in reader]

# データを格納するリスト
all_company_names = []
all_company_urls = []
all_extracted_links = []

# 各URLを訪問してデータを抽出
for url in urls:
    print(f"Processing URL: {url}")
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            
            # 会社名とURLを抽出
            company_names = []
            company_urls = []
            extracted_links = []
            
            companies = soup.find_all("div", class_="results__name")
            for company in companies:
                name = company.text.strip()
                parent = company.find_parent("a")
                company_url = parent.get("href") if parent else None
                if company_url and "http" not in company_url:
                    company_url = "https://biz-maps.com" + company_url
                
                # 各企業のページを訪問して特定リンクを取得
                if company_url:
                    try:
                        company_response = requests.get(company_url, headers=headers)
                        if company_response.status_code == 200:
                            company_soup = BeautifulSoup(company_response.content, "html.parser")
                            specific_links = company_soup.find_all("a", href=True)
                            external_links = [
                                link.get("href")
                                for link in specific_links
                                if "http" in link.get("href") and "biz-maps.com" not in link.get("href")
                            ]
                            
                            external_links = [
                                "" if link == "https://www.hifcorp.co.jp/" else link
                                for link in external_links
                            ]
                            
                            link_text = external_links[0] if external_links else "None"
                        else:
                            link_text = "Failed to fetch company page"

                    except Exception as e:
                        link_text = f"Error: {e}"
                else:
                    link_text = "None"
                
                company_names.append(name)
                company_urls.append(company_url)
                extracted_links.append(link_text)
            
                time.sleep(5)  # サーバー負荷軽減のため遅延を追加
            
            # URLごとのデータを統
            all_company_names.extend(company_names)
            all_company_urls.extend(company_urls)
            all_extracted_links.extend(extracted_links)
        else:
            print(f"Failed to fetch {url}: {response.status_code}")
    except Exception as e:
        print(f"Error processing {url}: {e}")

# 全データをCSV形式で保存
data = {
    "企業名": all_company_names,
    "企業URL": all_extracted_links
}
df = pd.DataFrame(data)
df.to_csv(output_csv, index=False, encoding="utf-8-sig")

# 終了時刻の記録と経過時間の計算
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Data saved to {output_csv}")
print(f"Total processing time: {elapsed_time:.2f} seconds")

Processing URL: https://biz-maps.com/s/cities/232017,232025,232033,232041,232050,232068,232076,232084,232092,232106,232114,232122,232131,232149,232157,232165,232173,232190,232203,232211,232220,232238,232246,232254,232262,232271,232289,232297,232301,232319,232327,232335,232343,232351,232360,232378,232386,233021,233421,233617,233625,234249,234257,234273,234419,234427,234451,234460,234478,235016,235610,235628,235636/b-inds/79
Processing URL: https://biz-maps.com/s/cities/232017,232025,232033,232041,232050,232068,232076,232084,232092,232106,232114,232122,232131,232149,232157,232165,232173,232190,232203,232211,232220,232238,232246,232254,232262,232271,232289,232297,232301,232319,232327,232335,232343,232351,232360,232378,232386,233021,233421,233617,233625,234249,234257,234273,234419,234427,234451,234460,234478,235016,235610,235628,235636/b-inds/79?url=2&city%5B0%5D=232017&city%5B1%5D=232025&city%5B2%5D=232033&city%5B3%5D=232041&city%5B4%5D=232050&city%5B5%5D=232068&city%5B6%5D=232076&city%5B